In [ ]:
%pip install openai
from openai import OpenAI
from pathlib import Path
import pandas as pd
import numpy as np
import os

In [ ]:
client = OpenAI(
    api_key = "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",
    base_url = "https://openrouter.ai/api/v1",
)

In [ ]:
def get_context(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        file = f.read()
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": "Read me this file of my java springboot application in the variable " + file + " and describe the functionality and context of the code written in the file"
            }
        ],
        model="deepseek/deepseek-r1-distill-llama-70b:free",
        stream=False,
    )
    return chat_completion.choices[0].message.content

In [ ]:
controllers = os.listdir("../code/src/main/java/net/engineeringdigest/journalApp/controller")
entities = os.listdir("../code/src/main/java/net/engineeringdigest/journalApp/entity")
repositories = os.listdir("../code/src/main/java/net/engineeringdigest/journalApp/repository")
services = os.listdir("../code/src/main/java/net/engineeringdigest/journalApp/service")
app_path = "../code/src/main/java/net/engineeringdigest/journalApp/JournalApplication.java"
context_path = "../code/src/main/resources/application.properties"

context_output_path = "../context_files/controllers_context.txt"
functionality_output_path = "../context_files/functionalities.txt"
tests_output_path = "../context_files/test_cases.txt"

In [ ]:
context_output = "CONTROLLER CONTEXT\n\n"
for controller in controllers:
    context_output += "CONTROLLER: " + controller + "\n\n"
    context_output += get_context("../code/src/main/java/net/engineeringdigest/journalApp/controller/" + controller) + "\n\n"

context_output += "ENTITY CONTEXT\n\n"
for entity in entities:
    context_output += "ENTITY: " + entity + "\n\n"
    context_output += get_context("../code/src/main/java/net/engineeringdigest/journalApp/entity/" + entity) + "\n\n"

context_output += "REPOSITORY CONTEXT\n\n"
for repository in repositories:
    context_output += "REPOSITORY: " + repository + "\n\n"
    context_output += get_context("../code/src/main/java/net/engineeringdigest/journalApp/repository/" + repository) + "\n\n"

context_output += "SERVICE CONTEXT\n\n"
for service in services:
    context_output += "SERVICE: " + service + "\n\n"
    context_output += get_context("../code/src/main/java/net/engineeringdigest/journalApp/service/" + service) + "\n\n"

context_output += "APPLICATION CONTEXT\n\n"
context_output += get_context(app_path) + "\n\n"

context_output += "APPLICATION PROPERTIES CONTEXT\n\n"
context_output += get_context(context_path) + "\n\n"

In [ ]:
open(context_output_path, "w", encoding="utf-8").write(context_output)

In [ ]:
chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": "Read me this file the code context and file structure of my java springboot project application in the variable " + context_output + " and make a list of all functionalties and map each functionality to all the files that have context for it/implements it"
            }
        ],
        model="deepseek/deepseek-r1-distill-llama-70b:free",
        stream=False,
    )
open(functionality_output_path, "w", encoding="utf-8").write(chat_completion.choices[0].message.content)

In [ ]:
chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": "Read me this file with the code context and file structure of my java springboot project application in the variable " + context_output + " and make a list of all functionalties. Then write functional test cases for each functionality including success and failure scenarios and all edge cases in the format of step_definition files as well as .feature files containing the scenarios. I need it in the Cucumber format with complete working implementations of the step_definition files and feature files with all scenarios enlisted under example tables."
            }
        ],
        model="deepseek/deepseek-r1-distill-llama-70b:free",
        stream=False,
        temperature=0
    )
open(tests_output_path, "w", encoding="utf-8").write(chat_completion.choices[0].message.content)

In [ ]:
import re

# Define the base directory for the test cases
base_dir = "../test_cases"
features_dir = os.path.join(base_dir, "features")
step_definitions_dir = os.path.join(features_dir, "step_definitions")

# Ensure the directories exist
os.makedirs(features_dir, exist_ok=True)
os.makedirs(step_definitions_dir, exist_ok=True)

# Read the content of the text file
with open("../context_files/test_cases.txt", "r", encoding="utf-8") as file:
    content = file.read()

# Regex patterns to extract feature files and step definitions
feature_pattern = r"\*\*File:\*\* `(.*?)`\n\n```gherkin\n(.*?)```"
step_def_pattern = r"\*\*Step Definition:\*\* `(.*?)`\n\n```java\n(.*?)```"

# Extract and create feature files
for match in re.finditer(feature_pattern, content, re.DOTALL):
    feature_file_name = match.group(1)
    feature_content = match.group(2)
    feature_file_path = os.path.join(features_dir, feature_file_name)

    # Write the feature file
    with open(feature_file_path, "w", encoding="utf-8") as feature_file:
        feature_file.write(feature_content)
    print(f"Created feature file: {feature_file_path}")

# Extract and create step definition files
for match in re.finditer(step_def_pattern, content, re.DOTALL):
    step_def_file_name = match.group(1)
    step_def_content = match.group(2)
    step_def_file_path = os.path.join(step_definitions_dir, step_def_file_name)

    # Write the step definition file
    with open(step_def_file_path, "w", encoding="utf-8") as step_def_file:
        step_def_file.write(step_def_content)
    print(f"Created step definition file: {step_def_file_path}")

print("All files have been created successfully.")